In [1]:
import os

import matplotlib.pyplot as plt
import open3d as o3d
import numpy as np
import shutil
import tqdm

from pathlib import Path

from python.plane_seg.algorithm.PEAC import PEAC
from python.plane_seg.metrics import evaluate_metrics

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Depth readers

In [2]:
from scripts.read_living import read_depth as read_living
from scripts.read_office import read_depth as read_office
from scripts.read_tum import read_depth as read_tum

## Constants

In [3]:
VALIDATION_FRAMES_PATH = {
    "icl_living_room": Path("/home/achains/Desktop/planeseg-metrics/validation_frames/icl_living_room/validation"),
    "icl_office": Path("/home/achains/Desktop/planeseg-metrics/validation_frames/icl_office/validation"),
    "tum_cabinet": Path("/home/achains/Desktop/planeseg-metrics/validation_frames/tum_cabinet/validation"),
    "tum_long_office": Path("/home/achains/Desktop/planeseg-metrics/validation_frames/tum_long_office/validation")
}

GROUND_TRUTH_PATH = {
    "icl_living_room": Path("/home/achains/Desktop/datasets/_validation_/icl-living-room_cc/output_cc"),
    "icl_office": Path("/home/achains/Desktop/datasets/_validation_/icl-office/output"),
    "tum_cabinet": Path("/home/achains/Desktop/datasets/_validation_/tum_freiburg3_cabinet/output"),
    "tum_long_office": Path("/home/achains/Desktop/datasets/_validation_/tum_long_office/output")
}

CFG_PATH = {
    "icl_living_room": Path("configs/PEAC/peac-living.ini"),
    "icl_office": Path("configs/PEAC/peac-office.ini"),
    "tum_cabinet": Path("configs/PEAC/peac-cabinet.ini"),
    "tum_long_office": Path("configs/PEAC/peac-long.ini")
}

DEFAULT_CFG = Path("configs/PEAC/peac_default.ini")

DATASET_READERS = {
    "icl_living_room": read_living,
    "icl_office": read_office,
    "tum_cabinet": read_tum,
    "tum_long_office": read_tum
}

## Evaluate Metrics

In [4]:
def evaluate_algorithm(cfg_path, pcd_path):
    algorithm = PEAC(
        container_name="peac_raw:1.0",
        cfg_path=cfg_path,
        pcd_path=pcd_path,
        size=(480, 640)
    )
    return algorithm.run()

In [5]:
def run_metrics(result_path, default_cfg_path=None):
    datasets = VALIDATION_FRAMES_PATH.keys()
    for ds_name in tqdm.tqdm(datasets):
        ds_output = Path(result_path) / Path(ds_name)
        if ds_output.exists():
            shutil.rmtree(ds_output)
        os.mkdir(ds_output)

        config_path = default_cfg_path or CFG_PATH[ds_name]
        depth_names = sorted(os.listdir(VALIDATION_FRAMES_PATH[ds_name]))

        if ds_name == "icl_office":
            depth_names = sorted(filter(lambda x: x.endswith(".depth"), depth_names))

        ds_metrics = {}
        for d_frame in depth_names:
            gt_frame_path = GROUND_TRUTH_PATH[ds_name] / (Path(d_frame).stem + ".png")
            pcd = DATASET_READERS[ds_name](str(VALIDATION_FRAMES_PATH[ds_name] / d_frame))
            o3d.io.write_point_cloud("tmp.ply", pcd)
            labels = evaluate_algorithm(
                cfg_path=config_path,
                pcd_path=Path("tmp.ply")
            )

            frame_metrics = evaluate_metrics(
                prediction=labels,
                ground_truth_path=gt_frame_path,
                print_to_console=False
            )
            for metric_name, metric_value in frame_metrics.items():
                if metric_name in ['iou', 'dice']:
                    continue
                if metric_name not in ds_metrics:
                    ds_metrics[metric_name] = []
                ds_metrics[metric_name].append(metric_value)

        for metric_name, value_array in ds_metrics.items():
            np.save(str(ds_output / metric_name), np.asarray(value_array))
        shutil.copy(config_path, ds_output)

## Result interpretation

In [6]:
import matplotlib.pyplot as plt

In [37]:
from matplotlib.pyplot import figure

In [13]:
METRIC_RESULTS = {
    "DDPFF": {
    "default": Path("/home/achains/Desktop/planeseg-metrics/DDPFF/results/default_params"),
    "custom": Path("/home/achains/Desktop/planeseg-metrics/DDPFF/results/my_params")
    },
    "CAPE": {
    "default": Path("/home/achains/Desktop/planeseg-metrics/CAPE/validation_metrics/default_params"),
    "custom": Path("/home/achains/Desktop/planeseg-metrics/CAPE/validation_metrics/my_params")
    },
    "PEAC": {
    "default": Path("/home/achains/Desktop/planeseg-metrics/PEAC/validation_metrics/default_params"),
    "custom": Path("/home/achains/Desktop/planeseg-metrics/PEAC/validation_metrics/custom_params")
    }
}

METRIC_NAMES = list(filter(
                    lambda fname: fname.endswith(".npy"),
                    os.listdir(METRIC_RESULTS["PEAC"]["custom"] / "icl_living_room")))

In [42]:
def fill_values(algorithm, ds_name, fun, conf="custom"):
    algorithm_values = {metric_name: 0.0 for metric_name in METRIC_NAMES}
    for metric_name in METRIC_NAMES:
        metric_values = np.load(METRIC_RESULTS[algorithm][conf] / ds_name / metric_name)
        algorithm_values[metric_name] = fun(metric_values)
    return algorithm_values

def dataset_evaluation(ds_name, fun, save_path=os.curdir):
    """
    ;
    :param ds_name: See VALIDATION_FRAMES_PATH.keys()
    :param fun: Statistic function
    :param save_path: Save .png
    """
    ALG_CONF = "custom"

    fig, axes = plt.subplots(3, 3, figsize=(15, 15))

    algorithm_values = {
        algorithm_name: fill_values(algorithm_name, ds_name, fun, conf=ALG_CONF) for algorithm_name in METRIC_RESULTS
    }
    for i, metric_name in enumerate(METRIC_NAMES):
        axes[i // 3][i % 3].set_ylim([0.0, 1.0])
        labels = sorted(algorithm_values.keys())
        values = [algorithm_values[algo_name][metric_name] for algo_name in sorted(algorithm_values.keys())]
        barlist = axes[i // 3][i % 3].bar(labels, values)

        barlist[0].set_color('r')
        barlist[1].set_color('g')
        axes[i // 3][i % 3].set_title(metric_name.rstrip(".npy"))
        for ax in axes.flat:
            ax.label_outer()

    fig.suptitle(ds_name, fontweight="bold")
    fig.tight_layout()
    fig.savefig(f"{ds_name}.jpg")

In [43]:
%%capture
dataset_evaluation("icl_living_room", np.mean)
dataset_evaluation("icl_office", np.mean)
dataset_evaluation("tum_cabinet", np.mean)
dataset_evaluation("tum_long_office", np.mean)